In [1]:
import pandas as pd


In [2]:
sales_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/f_sales_v_fashion.csv')
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['item'] = sales_data['sku'].astype(str)
sales_data['store'] = sales_data['store'].astype(str)
sales_data = sales_data.rename(columns={'total_sales':'sales'})
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990977 entries, 0 to 4990976
Data columns (total 7 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   sku                      int64         
 1   store                    object        
 2   date                     datetime64[ns]
 3   average_price            float64       
 4   average_dicounted_price  float64       
 5   sales                    float64       
 6   item                     object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 266.5+ MB


In [3]:
warehouse_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/warehouse_stock_fashion.csv')
warehouse_data['valid_to_date'] = warehouse_data['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))

In [4]:
warehouse_data['valid_to_date'].max()

Index(['sku', 'store', 'date', 'average_price', 'average_dicounted_price',
       'sales', 'item', 'sku_store'],
      dtype='object')

In [5]:
mbew_fashion = pd.read_csv('mbew_fashion.csv')
mbew_fashion['valid_to_date'] = mbew_fashion['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))
mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
mbew_fashion['item'] = mbew_fashion['sku'].astype(str).str[:12]
mbew_fashion

In [6]:
mbew_fashion_no_zero = mbew_fashion[mbew_fashion['stock'] > 0]
mbew_fashion_no_zero['valid_from_date'] = pd.to_datetime(mbew_fashion_no_zero['valid_from_date'])
dict_of_first_month_date_for_sku = {}
i= 0
for sku in mbew_fashion_no_zero['sku'].unique():
    i+=1
    print(str(i) + ' out of ' + str(len(mbew_fashion_no_zero['sku'].unique())))
    data_ = mbew_fashion_no_zero[mbew_fashion_no_zero['sku'] == sku]
    first_date = data_['valid_from_date'].min()
    # put the month in the dict from the first date
    dict_of_first_month_date_for_sku[sku] =  first_date.strftime('%m/%y')

       store              sku valid_from_date valid_to_date  stock
252746  VZ01  100550173000003      2018-02-16    2018-03-13  336.0
252867  VZ01  100550173000003      2018-03-14    2018-03-14  226.0
252986  VZ01  100550173000003      2018-03-15    2018-03-15  109.0
253153  VZ01  100550173000003      2018-03-16    2018-03-16   85.0
253330  VZ01  100550173000003      2018-03-17    2018-03-19   79.0
253491  VZ01  100550173000003      2018-03-20    2018-03-20   72.0
253635  VZ01  100550173000003      2018-03-21    2018-03-21   69.0
253800  VZ01  100550173000003      2018-03-22    2018-03-22   66.0
253882  VZ01  100550173000003      2018-03-23    2018-03-26   65.0
172706  VZ01  100550173000003      2018-03-27    2018-03-27   62.0
248197  VZ01  100550173000003      2018-03-28    2018-03-28   56.0
251470  VZ01  100550173000003      2018-03-29    2018-03-29   43.0
252646  VZ01  100550173000003      2018-03-30    2018-03-30   40.0
253228  VZ01  100550173000003      2018-03-31    2018-04-03   

In [6]:
# take MM/YY from first date
first_date.strftime('%m/%y')


In [7]:
dict_of_first_day_for_sku

,store,sku,valid_from_date,valid_to_date,stock,item
0,123,100549055000004,2017-04-21,2023-12-12,0.0,100549055000
1,89,100532175000001,2017-04-21,2023-12-12,0.0,100532175000
2,43,100553018000001,2017-04-21,2023-12-12,0.0,100553018000
3,104,100548169000007,2017-04-21,2023-12-12,0.0,100548169000
4,85,100549049000003,2017-04-21,2023-12-12,0.0,100549049000
...,...,...,...,...,...,...
17812117,18,100552850000002,2020-05-07,2023-12-12,0.0,100552850000
17812118,99,100653096000013,2020-05-07,2020-08-14,1.0,100653096000
17812119,46,100557049000004,2018-11-17,2023-12-12,0.0,100557049000
17812120,8,100511203000004,2017-04-21,2023-12-12,0.0,100511203000


In [8]:
print(sales_data)

In [9]:
strong_sku = sales_data.groupby(['sku'])['sales'].sum().sort_values(ascending=False).reset_index()

21925

In [16]:
import matplotlib.pyplot as plt
for sku in [100630490000001, 100630490000002, 100630490000003, 100630490000004, 100630490000005, 100630490000006, 100630490000007, 100630490000008, 100630490000009, 100630490000010, 100630490000011, 100630490000012, 100630490000013, 100630490000014, 100630490000015, 100630490000016, 100630490000017, 100630490000018, 100630490000019, 100630490000020, 100630490000021,]:
    sku_data = sales_data[sales_data['sku'] == sku].groupby('date')['sales'].sum()
    sku_data = sku_data.reindex(pd.date_range(start=sales_data['date'].min(),
                                                # today
                                                end=sales_data['date'].max(),

                                                freq='D')).fillna(0)
    sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')
    sku_warehouse_df_final = pd.DataFrame()
    for row in sku_warehouse.iterrows():
        sku_warehouse_df_final = pd.concat([sku_warehouse_df_final, pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'], end=row[1]['valid_to_date'], freq='D'), 'warehouse stock':row[1]['stock']})])
    sku_warehouse_df_final = sku_warehouse_df_final.set_index('date')
    relevant_sku_stores_stock = mbew_fashion[mbew_fashion['sku'] == sku]
    all_stores_sku_stock_data = {}
    for store in relevant_sku_stores_stock['store'].unique():
        store_sku_data = relevant_sku_stores_stock[relevant_sku_stores_stock['store'] == store]
        one_store_stock_sku_all = pd.DataFrame()
        for row in store_sku_data.iterrows():
            one_store_stock_sku = pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'],
                                                                     end=row[1]['valid_to_date'],
                                                                     freq='D'),
                                                'store stock':row[1]['stock']})
            one_store_stock_sku_all = pd.concat([one_store_stock_sku_all, one_store_stock_sku])
        one_store_stock_sku_all = one_store_stock_sku_all.set_index('date')
        all_stores_sku_stock_data[store] = one_store_stock_sku_all
        # merge all stores with outer join
    all_stores_final_stock_data = pd.DataFrame()
    for store in all_stores_sku_stock_data.keys():
        relevant_store_data_1 = all_stores_sku_stock_data[store]
        relevant_store_data_1 = relevant_store_data_1.rename(columns={'store stock':'store {}'.format(store) + ' stock'})
        all_stores_final_stock_data = pd.concat([all_stores_final_stock_data, relevant_store_data_1], axis=1, join='outer')
    all_store_sum_stock = all_stores_final_stock_data.sum(axis=1).to_frame().rename(columns={0:'all stores stock'})
    # merge by index
    sku_data = pd.merge(sku_data, sku_warehouse_df_final, left_index=True, right_index=True, how='left').fillna(0)
    sku_data = pd.merge(sku_data, all_store_sum_stock, left_index=True, right_index=True, how='left').fillna(0)
    sku_data_w = sku_data.resample('W').agg({'sales':'sum', 'warehouse stock':'mean', 'all stores stock':'mean'})
    sku_data_w.plot(figsize=(20,4))
    plt.title('sku: ' + str(sku))

    plt.show()

1/21925
2/21925
3/21925
4/21925
5/21925
6/21925
7/21925
8/21925
9/21925
10/21925
11/21925
12/21925
13/21925
14/21925
15/21925
16/21925
17/21925
18/21925
19/21925
20/21925
21/21925
22/21925
23/21925
24/21925
25/21925
26/21925
27/21925
28/21925
29/21925
30/21925
31/21925
32/21925
33/21925
34/21925
35/21925
36/21925
37/21925
38/21925
39/21925
40/21925
41/21925
42/21925
43/21925
44/21925
45/21925
46/21925
47/21925
48/21925
49/21925
50/21925
51/21925
52/21925
53/21925
54/21925
55/21925
56/21925
57/21925
58/21925
59/21925
60/21925
61/21925
62/21925
63/21925
64/21925
65/21925
66/21925
67/21925
68/21925
69/21925
70/21925
71/21925
72/21925
73/21925
74/21925
75/21925
76/21925
77/21925
78/21925
79/21925
80/21925
81/21925
82/21925
83/21925
84/21925
85/21925
86/21925
87/21925
88/21925
89/21925
90/21925
91/21925
92/21925
93/21925
94/21925
95/21925
96/21925
97/21925
98/21925
99/21925
100/21925
101/21925
102/21925
103/21925
104/21925
105/21925
106/21925
107/21925
108/21925
109/21925
110/21925
111/2192

In [ ]:
sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')

store: 76


In [33]:
sku

                 sku store       date  average_price  average_dicounted_price  \
0    100531078000001    76 2018-03-27            NaN                      NaN   
1    100611053000003    76 2023-05-13            NaN                      NaN   
2    100611053000003    76 2023-05-14            NaN                      NaN   
3    100611053000003    76 2023-05-15            NaN                      NaN   
4    100611053000003    76 2023-05-16            NaN                      NaN   
5    100611053000003    76 2023-05-17            NaN                      NaN   
6    100611053000003    76 2023-05-18            NaN                      NaN   
7    100611053000003    76 2023-05-19            NaN                      NaN   
8    100611053000003    76 2023-05-20            NaN                      NaN   
9    100611053000003    76 2023-05-21            NaN                      NaN   
10   100611053000003    76 2023-05-22            NaN                      NaN   
11   100611053000003    76 2

In [29]:
print(sku_warehouse.iloc[60:90])

In [30]:
3warehouse_data.info()


KeyboardInterrupt



In [ ]:
sku_warehouse_df_final

In [ ]:
print(100630490000005)

In [ ]:
sku_data